In [2]:
# pip install beautifulsoup4 requests


In [3]:
"""
pagerank_web.py
Implements the PageRank algorithm using BeautifulSoup and Python.
"""

'\npagerank_web.py\nImplements the PageRank algorithm using BeautifulSoup and Python.\n'

In [4]:
from bs4 import BeautifulSoup
import requests
import numpy as np

In [5]:
# ---------- CONFIGURATION ----------
# You can replace these with any small set of real URLs
URLS = [
    "https://en.wikipedia.org/wiki/Web_crawler",
    "https://en.wikipedia.org/wiki/PageRank",
    "https://en.wikipedia.org/wiki/Search_engine_optimization",
]
DAMPING_FACTOR = 0.85
MAX_ITER = 100
TOL = 1.0e-6
# -----------------------------------

In [9]:
def extract_links(url, all_urls):
    """
    Extracts outgoing links from a webpage that also belong to our URL set.
    """
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        links = set()
        for tag in soup.find_all("a", href=True):
            href = tag["href"]
            # Keep only internal Wikipedia links from our URL set
            if href.startswith("/wiki/") and not any(prefix in href for prefix in [":", "#"]):
                full_url = "https://en.wikipedia.org" + href
                if full_url in all_urls:
                    links.add(full_url)
        return links
    except Exception as e:
        print(f"[WARN] Failed to fetch {url}: {e}")
        return set()


In [10]:
def build_graph(urls):
    """
    Builds an adjacency dictionary {url: [linked_urls]}.
    """
    graph = {}
    for url in urls:
        print(f"Fetching links from: {url}")
        graph[url] = extract_links(url, urls)
    return graph


In [11]:
def page_rank(graph, damping=0.85, max_iter=100, tol=1e-6):
    """
    Computes PageRank using the iterative algorithm:
    PR(i) = (1 - d)/N + d * sum(PR(j)/outdeg(j)) for all j linking to i
    """
    nodes = list(graph.keys())
    N = len(nodes)
    ranks = np.ones(N) / N
    index = {nodes[i]: i for i in range(N)}

    # Build adjacency matrix
    adj = np.zeros((N, N))
    for src, out_links in graph.items():
        if len(out_links) == 0:
            adj[:, index[src]] = 1.0 / N  # Dangling node
        else:
            for dst in out_links:
                if dst in index:
                    adj[index[dst], index[src]] = 1.0 / len(out_links)

    # Iterate until convergence
    for it in range(max_iter):
        new_ranks = (1 - damping) / N + damping * adj @ ranks
        if np.linalg.norm(new_ranks - ranks, 1) < tol:
            print(f"✅ Converged after {it+1} iterations")
            break
        ranks = new_ranks

    return {nodes[i]: ranks[i] for i in range(N)}


In [12]:
print("🔍 Building link graph...")
graph = build_graph(URLS)

print("\n📊 Graph Structure:")
for k, v in graph.items():
    print(f"{k} → {list(v)}")

print("\n⚙️ Running PageRank...")
ranks = page_rank(graph, DAMPING_FACTOR, MAX_ITER, TOL)

print("\n🏆 === PageRank Scores ===")
for url, score in sorted(ranks.items(), key=lambda x: x[1], reverse=True):
    print(f"{url:80s} : {score:.4f}")


🔍 Building link graph...
Fetching links from: https://en.wikipedia.org/wiki/Web_crawler


C:\Users\SHREYAS\AppData\Local\Temp\ipykernel_20204\172323575.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(res.text, "html.parser")


Fetching links from: https://en.wikipedia.org/wiki/PageRank
Fetching links from: https://en.wikipedia.org/wiki/Search_engine_optimization

📊 Graph Structure:
https://en.wikipedia.org/wiki/Web_crawler → []
https://en.wikipedia.org/wiki/PageRank → []
https://en.wikipedia.org/wiki/Search_engine_optimization → []

⚙️ Running PageRank...
✅ Converged after 1 iterations

🏆 === PageRank Scores ===
https://en.wikipedia.org/wiki/Web_crawler                                        : 0.3333
https://en.wikipedia.org/wiki/PageRank                                           : 0.3333
https://en.wikipedia.org/wiki/Search_engine_optimization                         : 0.3333


In [13]:
# If you're offline or Wikipedia access fails, use this sample graph
toy_graph = {
    "A": {"B", "C"},
    "B": {"C"},
    "C": {"A"},
    "D": {"C"},
}

toy_ranks = page_rank(toy_graph)
print("\nToy Graph PageRank:")
for k, v in sorted(toy_ranks.items(), key=lambda x: x[1], reverse=True):
    print(f"{k} : {v:.4f}")


✅ Converged after 28 iterations

Toy Graph PageRank:
C : 0.3941
A : 0.3725
B : 0.1958
D : 0.0375
